# Codigo de Clase + Test Shorts

In [ ]:
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass, field
import empyrical
import optuna

In [ ]:
# --- 1. CONFIGURACIÓN Y CLASES DE DATOS ---

# Establece un estilo visual para los gráficos
sns.set_theme(style="whitegrid")

@dataclass
class Operation:
    """Clase para almacenar la información de una operación de trading."""
    open_time: pd.Timestamp
    open_price: float
    n_shares: int
    type: str  # 'LONG' o 'SHORT'
    stop_loss: float
    take_profit: float
    status: str = 'OPEN'
    close_time: pd.Timestamp = None
    close_price: float = None
    pnl: float = 0.0

In [ ]:
# --- 2. FUNCIONES DE CÁLCULO DE INDICADORES Y SEÑALES ---

def calculate_indicators(df: pd.DataFrame, params: dict) -> pd.DataFrame:
    """Calcula y añade los indicadores técnicos al DataFrame."""
    df_copy = df.copy()
    df_copy['rsi'] = ta.momentum.RSIIndicator(df_copy['Close'], window=params['rsi_window']).rsi()

    bollinger = ta.volatility.BollingerBands(df_copy['Close'], window=params['bb_window'], window_dev=2)
    df_copy['bb_high'] = bollinger.bollinger_hband()
    df_copy['bb_low'] = bollinger.bollinger_lband()

    stochastic = ta.momentum.StochasticOscillator(
        df_copy['High'], df_copy['Low'], df_copy['Close'],
        window=params['stoch_window'], smooth_window=params['stoch_smooth_k']
    )
    df_copy['stoch_k'] = stochastic.stoch()

    return df_copy.dropna()

def generate_signals(df: pd.DataFrame, params: dict) -> pd.DataFrame:
    """Genera señales de compra y venta basadas en el consenso de los indicadores."""
    df_copy = df.copy()
    # Señales individuales
    rsi_buy = df_copy['rsi'] < params['rsi_buy_th']
    rsi_sell = df_copy['rsi'] > params['rsi_sell_th']

    bb_buy = df_copy['Close'] < df_copy['bb_low']
    bb_sell = df_copy['Close'] > df_copy['bb_high']

    stoch_buy = df_copy['stoch_k'] < params['stoch_buy_th']
    stoch_sell = df_copy['stoch_k'] > params['stoch_sell_th']

    # Consenso: al menos 2 de 3 indicadores deben coincidir
    df_copy['buy_signal'] = (rsi_buy.astype(int) + bb_buy.astype(int) + stoch_buy.astype(int)) >= 2
    df_copy['sell_signal'] = (rsi_sell.astype(int) + bb_sell.astype(int) + stoch_sell.astype(int)) >= 2

    return df_copy

In [ ]:
# --- 3. LÓGICA DEL BACKTESTING ---

def run_backtest(df: pd.DataFrame, initial_cash: float, n_shares: int, commission: float, sl_pct: float, tp_pct: float):
    """Ejecuta la simulación de trading (backtest) iterando a través de los datos."""
    cash = initial_cash
    active_positions: list[Operation] = []
    closed_trades_log: list[Operation] = []
    portfolio_history = []

    for timestamp, row in df.iterrows():
        current_price = row['Close']

        # 1. CERRAR POSICIONES (revisar SL y TP primero)
        for position in active_positions[:]: # Iterar sobre una copia
            if position.type == 'LONG' and (current_price >= position.take_profit or current_price <= position.stop_loss):
                cash += current_price * position.n_shares * (1 - commission)
                position.status = 'CLOSED'
                position.close_time = timestamp
                position.close_price = current_price
                position.pnl = (position.close_price - position.open_price) * position.n_shares - (position.open_price + position.close_price) * position.n_shares * commission
                closed_trades_log.append(position)
                active_positions.remove(position)

            elif position.type == 'SHORT' and (current_price <= position.take_profit or current_price >= position.stop_loss):
                cash -= current_price * position.n_shares * (1 + commission)
                position.status = 'CLOSED'
                position.close_time = timestamp
                position.close_price = current_price
                position.pnl = (position.open_price - position.close_price) * position.n_shares - (position.open_price + position.close_price) * position.n_shares * commission
                closed_trades_log.append(position)
                active_positions.remove(position)

        # 2. ABRIR NUEVAS POSICIONES
        cost_of_long = current_price * n_shares * (1 + commission)
        if row['buy_signal'] and cash >= cost_of_long:
            cash -= cost_of_long
            active_positions.append(Operation(
                open_time=timestamp, open_price=current_price, n_shares=n_shares, type='LONG',
                stop_loss=current_price * (1 - sl_pct), take_profit=current_price * (1 + tp_pct)
            ))

        elif row['sell_signal']:
            cash += current_price * n_shares * (1 - commission)
            active_positions.append(Operation(
                open_time=timestamp, open_price=current_price, n_shares=n_shares, type='SHORT',
                stop_loss=current_price * (1 + sl_pct), take_profit=current_price * (1 - tp_pct)
            ))

        # 3. Actualizar valor del portafolio
        long_value = sum(p.n_shares * current_price for p in active_positions if p.type == 'LONG')
        short_liability = sum(p.n_shares * current_price for p in active_positions if p.type == 'SHORT')
        portfolio_value = cash + long_value - short_liability
        portfolio_history.append({'timestamp': timestamp, 'value': portfolio_value})

    # Cerrar todas las posiciones al final
    last_price = df['Close'].iloc[-1]
    for pos in active_positions:
        if pos.type == 'LONG': cash += last_price * pos.n_shares * (1 - commission)
        elif pos.type == 'SHORT': cash -= last_price * pos.n_shares * (1 + commission)

    if portfolio_history:
        portfolio_history[-1]['value'] = cash

    return pd.DataFrame(portfolio_history).set_index('timestamp')['value'], closed_trades_log

In [ ]:
# --- 4. MÉTRICAS DE DESEMPEÑO Y VISUALIZACIÓN ---

def calculate_performance_metrics(portfolio_values: pd.Series, trades_log: list[Operation]) -> dict:
    """Calcula las métricas clave de desempeño de la estrategia."""
    if portfolio_values.empty or len(trades_log) == 0:
        return {
            'Calmar Ratio': -1, 'Sharpe Ratio': 0, 'Sortino Ratio': 0,
            'Max Drawdown': -1, 'Win Rate': 0, 'Total Trades': 0
        }

    returns = portfolio_values.pct_change().dropna()

    # Asumiendo datos por hora -> 252 días de trading * 24 horas
    annualization_factor = 252 * 24

    metrics = {
        'Calmar Ratio': empyrical.calmar_ratio(returns, period='hourly'),
        'Sharpe Ratio': empyrical.sharpe_ratio(returns, period='hourly'),
        'Sortino Ratio': empyrical.sortino_ratio(returns, period='hourly'),
        'Max Drawdown': empyrical.max_drawdown(returns),
        'Win Rate': sum(1 for trade in trades_log if trade.pnl > 0) / len(trades_log),
        'Total Trades': len(trades_log)
    }
    return metrics

def plot_portfolio(portfolio_values: pd.Series, title: str):
    """Grafica la evolución del valor del portafolio."""
    plt.figure(figsize=(15, 7))
    portfolio_values.plot()
    plt.title(title)
    plt.ylabel('Valor del Portafolio (USD)')
    plt.xlabel('Fecha')
    plt.show()

In [ ]:
# --- 5. FUNCIÓN OBJETIVO PARA OPTIMIZACIÓN ---

def objective(trial: optuna.trial.Trial, df_train: pd.DataFrame) -> float:
    """Función que Optuna intentará maximizar."""
    params = {
        'rsi_window': trial.suggest_int('rsi_window', 10, 50),
        'bb_window': trial.suggest_int('bb_window', 10, 50),
        'stoch_window': trial.suggest_int('stoch_window', 10, 50),
        'stoch_smooth_k': trial.suggest_int('stoch_smooth_k', 3, 10),
        'rsi_buy_th': trial.suggest_int('rsi_buy_th', 20, 40),
        'rsi_sell_th': trial.suggest_int('rsi_sell_th', 60, 80),
        'stoch_buy_th': trial.suggest_int('stoch_buy_th', 10, 30),
        'stoch_sell_th': trial.suggest_int('stoch_sell_th', 70, 90),
    }

    # Constantes del proyecto
    INITIAL_CASH = 10_000_000
    N_SHARES = 1 # Usar 1 BTC por operación como ejemplo
    COMMISSION = 0.00125
    SL_PCT = 0.05 # Stop Loss del 5%
    TP_PCT = 0.10 # Take Profit del 10%

    # Ejecutar backtest con los parámetros de esta prueba
    df_indicators = calculate_indicators(df_train, params)
    df_signals = generate_signals(df_indicators, params)
    portfolio_values, trades_log = run_backtest(df_signals, INITIAL_CASH, N_SHARES, COMMISSION, SL_PCT, TP_PCT)

    if portfolio_values.empty or len(trades_log) < 10: # Penalizar si no hay suficientes operaciones
        return -1.0

    metrics = calculate_performance_metrics(portfolio_values, trades_log)
    return metrics['Calmar Ratio']

In [ ]:
def main():
    """Función principal que orquesta todo el proceso."""

    # Cargar y preparar los datos
    data = pd.read_csv('Binance_BTCUSDT_1h.csv')  # Asegúrate de tener este archivo en el mismo directorio

    data['Date'] = pd.to_datetime(data['Date'])
    data = data.set_index('Date').sort_index()

    # Dividir los datos: 60% Train, 20% Test, 20% Validation
    train_size = int(len(data) * 0.6)
    test_size = int(len(data) * 0.2)

    df_train = data.iloc[:train_size]
    df_test = data.iloc[train_size : train_size + test_size]
    df_validation = data.iloc[train_size + test_size :]

    print(f"Datos cargados: {len(data)} velas")
    print(f"Set de Entrenamiento: {len(df_train)} velas")
    print(f"Set de Prueba: {len(df_test)} velas")
    print(f"Set de Validación: {len(df_validation)} velas")

    # --- Optimización con Optuna ---
    print("\nIniciando optimización de parámetros con Optuna...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, df_train), n_trials=50) # n_trials controla cuánto tiempo busca

    print(f"\nOptimización completada. Mejor Calmar Ratio: {study.best_value:.4f}")
    print("Mejores parámetros encontrados:")
    best_params = study.best_params
    print(best_params)

    # --- Evaluación final con los mejores parámetros en el set de prueba ---
    print("\nEvaluando la estrategia con los mejores parámetros en el set de prueba...")

    # Constantes del proyecto para la evaluación final
    INITIAL_CASH = 10_000_000
    N_SHARES = 1
    COMMISSION = 0.00125
    SL_PCT = 0.05
    TP_PCT = 0.10

    df_test_indicators = calculate_indicators(df_test, best_params)
    df_test_signals = generate_signals(df_test_indicators, best_params)

    portfolio_values, trades_log = run_backtest(df_test_signals, INITIAL_CASH, N_SHARES, COMMISSION, SL_PCT, TP_PCT)

    if not portfolio_values.empty:
        final_metrics = calculate_performance_metrics(portfolio_values, trades_log)

        print("\n--- Métricas de Desempeño (Set de Prueba) ---")
        for key, value in final_metrics.items():
            print(f"{key}: {value:.4f}")

        plot_portfolio(portfolio_values, "Evolución del Portafolio (Set de Prueba)")
    else:
        print("\nNo se realizaron operaciones en el set de prueba con los parámetros óptimos.")


if __name__ == "__main__":
    main()